In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains

import pandas as pd
import time

def down_scroll_to_last(driver, selector, max_count):
    last_height = 0
    try_count = 0
    scroll_count = 0

    if not max_count:
        max_count = 100
        
    while True:
        
        now_height = driver.execute_script(f'return document.querySelector("{selector}").scrollHeight;')
        if last_height == now_height:
            if try_count > 10:
                break
        else:
            try_count = 0
            scroll_count += 1
            if scroll_count > max_count:
                break
        last_height = now_height
        driver.execute_script(f'document.querySelector("{selector}").scrollTo(0,document.querySelector("{selector}").scrollHeight);')
        time.sleep(0.5)
        try_count += 1

In [2]:
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

url_lst = [
    "https://new.land.naver.com/rooms?ms=37.5624965,126.9155239,15&a=APT:OPST:ABYG:OBYG:GM:OR:VL:DDDGG:JWJT:SGJT:HOJT&e=RETAIL&aa=SMALLSPCRENT",
    "https://new.land.naver.com/rooms?ms=37.5542295,126.9329904,15&a=APT:OPST:ABYG:OBYG:GM:OR:VL:DDDGG:JWJT:SGJT:HOJT&e=RETAIL&aa=SMALLSPCRENT",
    "https://new.land.naver.com/rooms?ms=37.547076,126.9178199,16&a=APT:OPST:ABYG:OBYG:GM:OR:VL:DDDGG:JWJT:SGJT:HOJT&e=RETAIL&aa=SMALLSPCRENT",
    "https://new.land.naver.com/rooms?ms=37.5269387,126.8997311,15&a=APT:OPST:ABYG:OBYG:GM:OR:VL:DDDGG:JWJT:SGJT:HOJT&e=RETAIL&aa=SMALLSPCRENT",
    "https://new.land.naver.com/rooms?ms=37.5360935,126.8979715,15&a=APT:OPST:ABYG:OBYG:GM:OR:VL:DDDGG:JWJT:SGJT:HOJT&e=RETAIL&aa=SMALLSPCRENT",
    "https://new.land.naver.com/rooms?ms=37.5242499,126.9085716,15&a=APT:OPST:ABYG:OBYG:GM:OR:VL:DDDGG:JWJT:SGJT:HOJT&e=RETAIL&aa=SMALLSPCRENT",
    "https://new.land.naver.com/rooms?ms=37.5589925,126.9353508,15&a=APT:OPST:ABYG:OBYG:GM:OR:VL:DDDGG:JWJT:SGJT:HOJT&e=RETAIL&aa=SMALLSPCRENT"
]

price_lst = list()
spec1_lst = list()
spec2_lst = list()

for url in url_lst:    
    driver.get(url)
    driver.implicitly_wait(1000)
    down_scroll_to_last(driver, "div.item_list.item_list--article", 10000)

    price_elems = driver.find_elements(By.CSS_SELECTOR, 'div.item_inner span.price')
    spec_elems = driver.find_elements(By.CSS_SELECTOR, 'div.info_area p.line span.spec')

    for price in price_elems:
        price_lst.append(price.text)
    for index, spec in enumerate(spec_elems):
        if index % 2 == 0:
            spec1_lst.append(spec.text)
        else:
            spec2_lst.append(spec.text)
driver.close()

pd.DataFrame({"price":price_lst, "spec1":spec1_lst, "spec2":spec2_lst}).to_csv('room_price_spec.csv', index=False)